In [1]:
import pyspark
import findspark
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession

import pandas as pd

from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession.builder.master('local[*]').getOrCreate()
spark

23/04/19 20:07:32 WARN Utils: Your hostname, MacBook-Air-Evgenij.local resolves to a loopback address: 127.0.0.1; using 192.168.0.102 instead (on interface en0)
23/04/19 20:07:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/19 20:07:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = pd.read_csv('./train.csv')
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [4]:
df.fillna(0, inplace=True)

In [5]:
columns = ['price_doc', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq', 'state', 'product_type', 'sub_area']
df = df[columns]

In [6]:
df = df._get_numeric_data()
df.head()

,price_doc,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state
0,5850000,43,27.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6000000,34,19.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5700000,43,29.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13100000,89,50.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16331452,77,77.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df.to_csv('train_for_HW_l4.csv', index=False)

In [8]:
dataset = spark.read.csv('./train_for_HW_l4.csv', inferSchema=True, header=True)
dataset.printSchema()

root
 |-- price_doc: integer (nullable = true)
 |-- full_sq: integer (nullable = true)
 |-- life_sq: double (nullable = true)
 |-- floor: double (nullable = true)
 |-- max_floor: double (nullable = true)
 |-- material: double (nullable = true)
 |-- build_year: double (nullable = true)
 |-- num_room: double (nullable = true)
 |-- kitch_sq: double (nullable = true)
 |-- state: double (nullable = true)



In [9]:
dataset.show()

+---------+-------+-------+-----+---------+--------+----------+--------+--------+-----+
|price_doc|full_sq|life_sq|floor|max_floor|material|build_year|num_room|kitch_sq|state|
+---------+-------+-------+-----+---------+--------+----------+--------+--------+-----+
|  5850000|     43|   27.0|  4.0|      0.0|     0.0|       0.0|     0.0|     0.0|  0.0|
|  6000000|     34|   19.0|  3.0|      0.0|     0.0|       0.0|     0.0|     0.0|  0.0|
|  5700000|     43|   29.0|  2.0|      0.0|     0.0|       0.0|     0.0|     0.0|  0.0|
| 13100000|     89|   50.0|  9.0|      0.0|     0.0|       0.0|     0.0|     0.0|  0.0|
| 16331452|     77|   77.0|  4.0|      0.0|     0.0|       0.0|     0.0|     0.0|  0.0|
|  9100000|     67|   46.0| 14.0|      0.0|     0.0|       0.0|     0.0|     0.0|  0.0|
|  5500000|     25|   14.0| 10.0|      0.0|     0.0|       0.0|     0.0|     0.0|  0.0|
|  2000000|     44|   44.0|  5.0|      0.0|     0.0|       0.0|     0.0|     0.0|  0.0|
|  5300000|     42|   27.0|  5.0

In [10]:
dataset.columns

['price_doc',
 'full_sq',
 'life_sq',
 'floor',
 'max_floor',
 'material',
 'build_year',
 'num_room',
 'kitch_sq',
 'state']

In [11]:
input_col = ['full_sq',
 'life_sq',
 'floor',
 'max_floor',
 'material',
 'build_year',
 'num_room',
 'kitch_sq',
 'state']

In [12]:
assembler = VectorAssembler(inputCols=['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq', 'state'], outputCol='Attributes')

output = assembler.transform(dataset)

finalized_data = output.select('Attributes', 'price_doc')

finalized_data.show()

+--------------------+---------+
|          Attributes|price_doc|
+--------------------+---------+
|(9,[0,1,2],[43.0,...|  5850000|
|(9,[0,1,2],[34.0,...|  6000000|
|(9,[0,1,2],[43.0,...|  5700000|
|(9,[0,1,2],[89.0,...| 13100000|
|(9,[0,1,2],[77.0,...| 16331452|
|(9,[0,1,2],[67.0,...|  9100000|
|(9,[0,1,2],[25.0,...|  5500000|
|(9,[0,1,2],[44.0,...|  2000000|
|(9,[0,1,2],[42.0,...|  5300000|
|(9,[0,1,2],[36.0,...|  2000000|
|(9,[0,1,2],[36.0,...|  4650000|
|(9,[0,1,2],[38.0,...|  4800000|
|(9,[0,1,2],[43.0,...|  5100000|
|(9,[0,1,2],[31.0,...|  5200000|
|(9,[0,1,2],[31.0,...|  5000000|
|(9,[0,1,2],[51.0,...|  1850000|
|(9,[0,1,2],[47.0,...|  6300000|
|(9,[0,1,2],[42.0,...|  5900000|
|(9,[0,1,2],[59.0,...|  7900000|
|(9,[0,1,2],[44.0,...|  5200000|
+--------------------+---------+
only showing top 20 rows



In [13]:
train_data, test_data = finalized_data.randomSplit([0.8, 0.2])

regressor = LinearRegression(featuresCol='Attributes', labelCol='price_doc')

regressor = regressor.fit(train_data)

pred = regressor.evaluate(test_data)

pred.predictions.show()

23/04/19 20:07:39 WARN Instrumentation: [8649205f] regParam is zero, which might cause numerical instability and overfitting.
23/04/19 20:07:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/04/19 20:07:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/04/19 20:07:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


+--------------------+---------+------------------+
|          Attributes|price_doc|        prediction|
+--------------------+---------+------------------+
|      (9,[0],[51.0])|  4281587| 4453806.270854042|
|      (9,[0],[60.0])|  5753200| 4705147.671277579|
|      (9,[0],[93.0])|  5334133| 5626732.806163879|
|      (9,[0],[97.0])|  8056968| 5738440.095241006|
|     (9,[0],[100.0])| 14604042| 5822220.562048852|
|     (9,[0],[138.0])|  5624541| 6883439.808281562|
|     (9,[0],[159.0])|  7434966| 7469903.075936481|
|(9,[0,1],[18.0,38...|  5200000|3857601.3512517335|
|(9,[0,1],[25.0,10...|  2000000|  3813335.26429587|
|(9,[0,1],[25.0,10...|  4100000|  3813335.26429587|
|(9,[0,1],[32.0,15...|  4000000| 4051636.206402421|
|(9,[0,1],[34.0,19...|  4480000| 4141740.399918247|
|(9,[0,1],[34.0,20...|  2000000|4150303.0371625624|
|(9,[0,1],[34.0,20...|  5700000|4150303.0371625624|
|(9,[0,1],[35.0,19...|  1000000|4169667.2221875284|
|(9,[0,1],[38.0,19...|  6200000| 4253447.688995374|
|(9,[0,1],[3

In [14]:
coef = regressor.coefficients

intr = regressor.intercept

print('coef: %a' % coef)
print('intr: %a' % intr)

coef: DenseVector([27926.8223, 8562.6372, 125865.458, -66221.4032, -10128.4573, -0.0862, 1354675.8011, 3127.7431, 147784.3606])
intr: 3029538.335120668


In [15]:
eval = RegressionEvaluator(labelCol='price_doc', predictionCol='prediction', metricName='rmse')

In [16]:
rmse = eval.evaluate(pred.predictions)
print('rmse: %.3f' % rmse)

mse = eval.evaluate(pred.predictions, {eval.metricName:'mse'})
print('mse: %.3f' % mse)

mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("mae: %.3f" % mae)

r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" % r2)

rmse: 3809861.166
mse: 14515042107153.656
mae: 2346734.935
r2: 0.280


23/04/19 20:07:53 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
